<a href="https://colab.research.google.com/github/adrianaleticiamartinez/mcd_deep_learning/blob/main/EntrenamientoGeneradorCuentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# Importar las librerías necesarias
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
import torch.nn.functional as F
import base64
import requests

In [31]:
# Configurar el dispositivo para GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [60]:
# Parámetros del modelo
# ------------------------------
# n_hidden: Número de unidades en las capas ocultas.
n_hidden = 512

# n_layers: Número de capas en el modelo LSTM.
n_layers = 2

# batch_size: Número de ejemplos que se procesan en cada paso de entrenamiento
batch_size = 16
#batch_size = 10

# seq_length: Longitud de las secuencias de entrada (en términos de caracteres) que el modelo verá durante el entrenamiento.
seq_length = 100

# epochs: Número de veces que todo el conjunto de datos será utilizado para entrenar el modelo
epochs = 150
#epochs = 2

# lr (learning rate): Tasa de aprendizaje, determina el tamaño de los pasos que da el modelo para ajustar sus pesos
lr = 0.001

# clip: Valor para "recortar" los gradientes y prevenir que se vuelvan demasiado grandes, lo que puede causar inestabilidad en el entrenamiento.
clip = 5

# top_k: Número de predicciones más probables a considerar al generar nuevo texto. Valores bajos generan texto más "creativo", pero menos coherente.
top_k = 5


In [49]:
# Preprocesamiento del texto: convertir el corpus a caracteres únicos y mapearlos a índices
def preprocess_text(text):
    chars = tuple(set(text))
    int2char = dict(enumerate(chars))
    char2int = {ch: ii for ii, ch in int2char.items()}
    encoded = np.array([char2int[ch] for ch in text])
    return chars, int2char, char2int, encoded

In [50]:
# Definir el modelo LSTM
class LSTM(nn.Module):
    def __init__(self, chars, n_hidden=n_hidden, n_layers=n_layers, drop_prob=0.5):
        super(LSTM, self).__init__()
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        self.n_chars = len(chars)

        self.lstm = nn.LSTM(self.n_chars, n_hidden, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden, self.n_chars)

    def forward(self, x, hidden):
        r_output, hidden = self.lstm(x, hidden)
        out = self.dropout(r_output)
        out = out.contiguous().view(-1, self.n_hidden)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().to(device))
        return hidden

In [51]:
# Función para one-hot encoder
def one_hot_encode(arr, n_labels):
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    return one_hot

In [52]:
# Crear batches para el entrenamiento
def get_batches(arr, batch_size, seq_length):
    batch_size_total = batch_size * seq_length
    n_batches = len(arr) // batch_size_total
    arr = arr[:n_batches * batch_size_total]
    arr = arr.reshape((batch_size, -1))

    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n:n+seq_length]
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y


In [61]:
# Función para entrenar el modelo
def train(model, data, epochs=epochs, batch_size=batch_size, seq_length=seq_length, lr=lr, clip=clip):
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
        h = model.init_hidden(batch_size)  # Inicializa el estado oculto al comienzo de cada época
        for x, y in get_batches(data, batch_size, seq_length):
            x = one_hot_encode(x, model.n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            inputs, targets = inputs.to(device), targets.to(device)

            # Desvincular el estado oculto de su historial de gradientes anterior
            h = tuple([each.detach() for each in h])

            model.zero_grad()  # Resetear los gradientes
            output, h = model(inputs, h)
            loss = criterion(output, targets.view(batch_size * seq_length).long())
            loss.backward()

            # Evitar la explosión de gradientes
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            optimizer.step()

        print(f"Epoch: {epoch+1}/{epochs}... Loss: {loss.item()}")



In [62]:
def predict(model, char, device, h=None, top_k=5):

    # Convertir el caracter a su índice entero correspondiente
    x = np.array([[model.char2int[char]]])

    # Codificar el índice entero en formato one-hot
    x = one_hot_encode(x, model.n_chars)

    # Convertir el array de NumPy a un tensor de PyTorch y moverlo al dispositivo especificado (CPU o GPU)
    inputs = torch.from_numpy(x).to(device)

    # Desactivar el cálculo del gradiente para la predicción
    with torch.no_grad():
        out, h = model(inputs, h)

        # Aplicar softmax para obtener las probabilidades de salida
        p = F.softmax(out, dim=1).data.cpu()

        # Obtener los top k caracteres más probables y sus probabilidades
        p, top_ch = p.topk(top_k)

        # Convertir los top k caracteres y probabilidades a arrays de NumPy
        top_ch = top_ch.numpy().squeeze()
        p = p.numpy().squeeze()

        # Elegir aleatoriamente el siguiente carácter de los top k caracteres basándonos en sus probabilidades
        char = np.random.choice(top_ch, p=p/p.sum())

    # Devolver el carácter predicho y el estado oculto actualizado
    return model.int2char[char], h

In [55]:
def sample3(model, size, device, prime='A', top_k=top_k):
    # Método para generar un nuevo texto basado en una secuencia inicial "prime".
    # Esencialmente, esta función es un ciclo que llama al método "predict" definido anteriormente.
    model.eval()  # Cambiar el modelo a modo de evaluación (eval mode)

    # Calcular el estado oculto del modelo utilizando los caracteres iniciales (prime)
    chars = [ch for ch in prime]  # Crear una lista de los caracteres iniciales
    with torch.no_grad():  # Desactivar el cálculo del gradiente para hacer predicciones
        # Inicializar el estado oculto con ceros al principio. El tamaño del batch es 1
        # ya que deseamos generar una sola secuencia de texto.
        h = model.init_hidden(batch_size=1)

        # Procesar cada carácter de la secuencia inicial prime
        for ch in prime:
            char, h = predict(model, ch, device, h=h, top_k=top_k)

        # Añadir el carácter generado a la secuencia
        chars.append(char)

        # Ahora, se toma el último carácter de la secuencia y se obtiene el siguiente.
        # Este proceso se repite segun lo especificado
        for ii in range(size):
            char, h = predict(model, chars[-1], device, h=h, top_k=top_k)
            chars.append(char)

    # Unir la lista de caracteres
    return ''.join(chars)

In [40]:
!wget https://raw.githubusercontent.com/adrianaleticiamartinez/mcd_deep_learning/refs/heads/main/cleaned_merged_fairy_tales_without_eos.txt

--2024-09-26 06:04:34--  https://raw.githubusercontent.com/adrianaleticiamartinez/mcd_deep_learning/refs/heads/main/cleaned_merged_fairy_tales_without_eos.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20635335 (20M) [text/plain]
Saving to: ‘cleaned_merged_fairy_tales_without_eos.txt’

cleaned_merged_fair 100%[===================>]  19.68M  --.-KB/s    in 0.05s   

2024-09-26 06:04:35 (374 MB/s) - ‘cleaned_merged_fairy_tales_without_eos.txt’ saved [20635335/20635335]



In [63]:
# Cargar el dataset y preprocesarlo


with open('cleaned_merged_fairy_tales_without_eos.txt', 'r') as f:
    text = f.read()

chars, int2char, char2int, encoded = preprocess_text(text)

In [64]:
# Configurar el modelo
model = LSTM(chars, n_hidden=n_hidden, n_layers=n_layers).to(device)
model.int2char = int2char
model.char2int = char2int

# Entrenar el modelo
train(model, encoded)

Epoch: 1/150... Loss: 1.370241641998291
Epoch: 2/150... Loss: 1.3183226585388184
Epoch: 3/150... Loss: 1.2729926109313965
Epoch: 4/150... Loss: 1.25300133228302
Epoch: 5/150... Loss: 1.2597562074661255
Epoch: 6/150... Loss: 1.2478523254394531
Epoch: 7/150... Loss: 1.24309241771698
Epoch: 8/150... Loss: 1.2236354351043701
Epoch: 9/150... Loss: 1.2116458415985107
Epoch: 10/150... Loss: 1.2217319011688232
Epoch: 11/150... Loss: 1.2309726476669312
Epoch: 12/150... Loss: 1.2315723896026611
Epoch: 13/150... Loss: 1.214572548866272
Epoch: 14/150... Loss: 1.2257651090621948
Epoch: 15/150... Loss: 1.2202945947647095
Epoch: 16/150... Loss: 1.2133337259292603
Epoch: 17/150... Loss: 1.2101211547851562
Epoch: 18/150... Loss: 1.2197320461273193
Epoch: 19/150... Loss: 1.206028938293457
Epoch: 20/150... Loss: 1.2119054794311523
Epoch: 21/150... Loss: 1.1954658031463623
Epoch: 22/150... Loss: 1.1985055208206177
Epoch: 23/150... Loss: 1.1853152513504028
Epoch: 24/150... Loss: 1.2157864570617676
Epoch: 2

In [65]:
# Guardar el modelo entrenado
model_save_path = "lstm_cuentos_infantiles_150epoch.pth"
torch.save(model.state_dict(), model_save_path)

In [66]:
# prime: Texto inicial con el que comienza la generación de nuevo texto.
# ---- Cambiar aqui para pruebas c: ----
prime = "Once upon a time"

In [67]:
# Generar un cuento
#generated_text = sample(model, 500)
generated_text = sample3(model, 1000, device, prime=prime)
print(generated_text)

Once upon a time; be on s ann, and stanehy nute dar bigelen hatterga  the ho s   And the statu shall of sin sheds was; and woe  tirs, that son of the crest, and ware  the tail would dee s  humtinge to break it and ooas. the sun sprint of then wound awah, wound ensito aetse t trees toh hra  h toinnu toe he ha st ahee,h ar,t hean had ene, drthy seoueth en as ans to gaysno a so s wastede a gooe rris e da  sa todurtoe ste woo ss ant
e   shhe  ouu at ohe a  he weatfn bacen beneernat. Thomethe ofe as ane drewe shou  tilkitt sea hee wine th breaphunente shh ahuthe  to min, as sakid he thouch. When he cise shel  wild by ant sharn.
 Thou are su daite wes so oonea tok to wilse teo dw fto ne site. 
T he had a goon heat oa. I sum the stead is no moute. If you done help hals, the midnans, a leet her on  tet ". de heer onst htha sene a sadisentont the ants  it a boom.  I am tiskfore a,  "That arl we," said the comair of liferaieec. “Woros to sen,” he said. “He is thie, and the other and of ath. “Wot